In [1]:
import numpy as np
import random
from wordPreprocess import WordPreprocess
from gensim.models import word2vec

In [2]:
programs_questions = []
num_lines = 0
with open('program_quesiton.txt', 'r') as inFile:
    for line in inFile:
        programs_questions.append(line.strip('\n'))
        num_lines += 1
        
        if num_lines % 100000 == 0:
            print ("Finished %d of lines" % num_lines)

word2vec_model = word2vec.Word2Vec.load('word2vec_model.bin')

Finished 100000 of lines
Finished 200000 of lines
Finished 300000 of lines
Finished 400000 of lines
Finished 500000 of lines
Finished 600000 of lines
Finished 700000 of lines
Finished 800000 of lines
Finished 900000 of lines
Finished 1000000 of lines
Finished 1100000 of lines
Finished 1200000 of lines
Finished 1300000 of lines
Finished 1400000 of lines
Finished 1500000 of lines
Finished 1600000 of lines
Finished 1700000 of lines
Finished 1800000 of lines
Finished 1900000 of lines
Finished 2000000 of lines
Finished 2100000 of lines
Finished 2200000 of lines
Finished 2300000 of lines
Finished 2400000 of lines


word_preprocess = WordPreprocess()
doc_hash = word_preprocess.hash_transform(programs_questions, num_feature=2**7)
print (doc_hash.shape)

In [3]:
doc_vec = []

for sentence in programs_questions:
    words = sentence.split(' ')[:-1]
    vec = np.zeros(shape=(250,), dtype=float)
    count = 0
    for w in words:
        try:
            vec += word2vec_model.wv[w]
            count += 1
        except KeyError:
            pass
        
    #if (count > 0):
    #    vec /= count
    doc_vec.append(vec)

In [4]:
programs = programs_questions[:2396906]
questions = programs_questions[2396906:]

programs_vec = doc_vec[:2396906]
questions_vec = doc_vec[2396906:]

In [5]:
def generate_training_data():
    Xs, Ys = [], []
    NUM_TRAIN = 500000
    itr = 0
    
    while (itr < NUM_TRAIN):
        pos_or_neg = random.randint(0,1)
        
        if pos_or_neg is 1:
            line_id = random.randint(0, len(programs)-1)
            
            L1 = programs[line_id].split(' ')[:-1]
            L2 = programs[line_id+1].split(' ')[:-1]
            
            if (not not L1 and not not L2):
                try:
                    Ys.append(word2vec_model.n_similarity(L1, L2))
                    Xs.append([programs_vec[line_id], programs_vec[line_id+1]])
                    itr += 1
                except KeyError:
                    pass
            
        else:
            line1_id = random.randint(0, len(programs)-1)
            line2_id = random.randint(0, len(programs)-1)
            
            L1 = programs[line1_id].split(' ')[:-1]
            L2 = programs[line2_id].split(' ')[:-1]
            
            if (not not L1 and not not L2):
                try:
                    Ys.append(word2vec_model.n_similarity(L1, L2))
                    Xs.append([programs_vec[line1_id], programs_vec[line2_id]])
                    itr += 1
                except KeyError:
                    pass
            
    return Xs, Ys

In [6]:
X, y = generate_training_data()

X_ = []


for t in X:
    X_.append(np.dot(t[0], t[1])*t[1]/np.linalg.norm(t[1]))

X_ = np.stack(X_, axis=0)
print (X_)

[[ -71.24503981  -39.82789333   -0.57092833 ...,  -15.1777363    19.54160349
     5.17705814]
 [ -31.39640256   -6.64013747  -25.63856923 ...,  -14.89299984
     8.25296802   33.90197016]
 [ -60.07967457   -8.35876344  -17.65124545 ...,  -37.0523163    24.38538967
     1.38797454]
 ..., 
 [ -59.95008327    3.16324217   37.43261593 ...,  -40.55259431    4.5161483
    50.96414012]
 [-110.90598182   -4.07486932  -30.11510022 ...,  -42.238529      8.9708259
   106.16224528]
 [ -60.70204079   41.25366257 -141.51711863 ..., -100.92373302
    26.05217959  123.96008004]]


In [18]:
X_ = []
for t in X:
    X_.append(np.dot(t[0], t[1])*t[0]/np.linalg.norm(t[0]))
    
X_ = np.stack(X_, axis=0)
y = np.array(y)

In [19]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sys

cv5 = KFold(n_splits=5, shuffle=True, random_state=1)

cv5_score = []
best_score = sys.float_info.max
best_i = -1
clfs = {}

for i, (train, valid) in enumerate(cv5.split(X_, y)):
    X_train, X_valid = X_[train], X_[valid]
    y_train, y_valid = y[train], y[valid]
    
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_valid_std = sc.transform(X_valid)
    
    lr = LinearRegression()
    lr.fit(X_train_std, y_train)
    
    y_pred_lr = lr.predict(X_valid_std)
    cv5_score.append(mean_squared_error(y_valid, y_pred_lr))
    clfs[i] = lr
    if mean_squared_error(y_valid, y_pred_lr) < best_score:
        best_score, best_i = mean_squared_error(y_valid, y_pred_lr), i
        
for j, score in enumerate(cv5_score):
    print ("[Fold %d|5]" % (j+1))
    print ("MSE: %.4f" % score)
    
print ("Best MSE: %.4f in %dth fold" % (best_score, best_i))

[Fold 1|5]
MSE: 0.0214
[Fold 2|5]
MSE: 0.0194
[Fold 3|5]
MSE: 0.0193
[Fold 4|5]
MSE: 0.0200
[Fold 5|5]
MSE: 0.0192
Best MSE: 0.0192 in 4th fold


from sklearn.model_selection import train_test_split

X, y = generate_training_data()

X_ = []

for t in X:
    X_.append(np.dot(t[0], t[1])*t[1]/np.linalg.norm(t[1]))
    

X_ = np.stack(X_, axis=0)
print (X_)

X_train, X_valid, y_train, y_valid = train_test_split(X_, y, test_size=0.3, random_state=1)

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_valid_std = sc.transform(X_valid)

## LinearRegression with jieba using big5_dict gets best accuracy so far, accuracy: 0.368

from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train_std, y_train)

y_pred = regr.predict(X_valid_std)
print("MSE: %f" % mean_squared_error(y_valid, y_pred))

from sklearn.linear_model import Ridge
lr_rg = Ridge(alpha=1000)
lr_rg.fit(X_train_std, y_train)
    
y_pred_rg = lr_rg.predict(X_valid_std)
    
print ("MSE: %f" % mean_squared_error(y_valid, y_pred_rg))

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_rbf = SVC(kernel='rbf', random_state=0, C=10.0, gamma=0.2)
svm_rbf.fit(X_train_std, y_train)

y_pred = svm_rbf.predict(X_valid_std)

print ("Accuracy: %.3f" % (accuracy_score(y_valid, y_pred)))

from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=10)

tree.fit(X_train_std, y_train)

y_pred_tree = tree.predict(X_valid_std)

print ("MSE: %f" % mean_squared_error(y_valid, y_pred_tree))

In [20]:
import sys

q_test = []
for q_id in range(500):
    question = questions_vec[q_id*7]
    answer = 0
    sim_max = sys.float_info.min
    for i in range(1,7):
        ans = questions_vec[q_id*7 + i]
        #set_ = [x+y for x, y in zip(question, ans)]
        set_ = np.dot(question, ans)*question/np.linalg.norm(question)
        #set_ = np.array(set_)
        result = clfs[best_i].predict(set_.reshape(1,-1))
        if result > sim_max:
            answer = i-1
            sim_max = result
        
    q_test.append((q_id, answer))

In [21]:
print (q_test)

[(0, 4), (1, 4), (2, 3), (3, 4), (4, 0), (5, 3), (6, 2), (7, 0), (8, 0), (9, 4), (10, 0), (11, 2), (12, 0), (13, 1), (14, 5), (15, 0), (16, 4), (17, 4), (18, 0), (19, 0), (20, 2), (21, 5), (22, 4), (23, 0), (24, 2), (25, 1), (26, 4), (27, 2), (28, 4), (29, 0), (30, 2), (31, 2), (32, 3), (33, 1), (34, 4), (35, 3), (36, 0), (37, 0), (38, 4), (39, 0), (40, 3), (41, 4), (42, 5), (43, 0), (44, 3), (45, 5), (46, 4), (47, 1), (48, 2), (49, 3), (50, 5), (51, 4), (52, 5), (53, 1), (54, 0), (55, 1), (56, 3), (57, 3), (58, 0), (59, 0), (60, 0), (61, 0), (62, 3), (63, 0), (64, 3), (65, 4), (66, 4), (67, 3), (68, 3), (69, 4), (70, 5), (71, 5), (72, 0), (73, 3), (74, 3), (75, 3), (76, 0), (77, 0), (78, 1), (79, 0), (80, 5), (81, 3), (82, 2), (83, 5), (84, 1), (85, 2), (86, 2), (87, 5), (88, 5), (89, 1), (90, 5), (91, 4), (92, 1), (93, 3), (94, 5), (95, 1), (96, 0), (97, 1), (98, 3), (99, 5), (100, 3), (101, 4), (102, 0), (103, 1), (104, 0), (105, 2), (106, 4), (107, 2), (108, 1), (109, 4), (110, 4),

In [22]:
import pandas as pd

df = pd.DataFrame(data=q_test, columns=['Id', 'Answer']).to_csv('Answer.csv', index=False)